In [1]:
import kwant
import numpy as np
import matplotlib.pyplot as plt
import tinyarray

sigma_0 = tinyarray.array([[1, 0], [0, 1]])
sigma_x = tinyarray.array([[0, 1], [1, 0]])
sigma_y = tinyarray.array([[0, -1j], [1j, 0]])
sigma_z = tinyarray.array([[1, 0], [0, -1]])

I_4= tinyarray.array(np.kron(sigma_0, sigma_0))
gamma_1 = tinyarray.array(np.kron(sigma_0, sigma_z))
gamma_2 = tinyarray.array(np.kron(sigma_z, sigma_z))
gamma_3 = tinyarray.array(np.kron(sigma_0, sigma_y))
gamma_4 = tinyarray.array(np.kron(sigma_0, sigma_x))

B=-300
g=300
las=1

structure=kwant.lattice.square(las,norbs=4)
model_sym= kwant.TranslationalSymmetry(structure.vec((-1,0)),structure.vec((0,-1)))

def onsite(site,m):
    return m/2*gamma_1 - (4*B)/las**2 * gamma_2

hopping_x = B/(las**2)*gamma_2 + 1j*g/(2*las)*gamma_3
hopping_y = B/(las**2)*gamma_2 - 1j*g/(2*las)*gamma_4

def model_MTI():
    model_1= kwant.Builder(model_sym)

    model_1[structure(0, 0)] = onsite
    model_1[kwant.builder.HoppingKind((1,0), structure)]  = hopping_x
    model_1[kwant.builder.HoppingKind((0,1), structure)]  = hopping_y

    return model_1

def model_EG():

    onsite_leads= 150 *I_4
    hopping_leads= -150 *I_4

    model_2= kwant.Builder(model_sym)
    model_2[structure(0, 0)] = onsite_leads
    model_2[structure.neighbors()]  = hopping_leads

    return model_2

def make_syst(width=50,length=100,leads_model=model_EG()):
    
    ########### create the scattering center

    def shape_center(site):
        x, y = site.pos
        return 0 < x <= length and 0 < y <= width

    syst= kwant.Builder()
    syst.fill(model_MTI(), shape_center,(1,1))

    ########### create the leads

    width_lead = length//5

    lead0= kwant.Builder(kwant.TranslationalSymmetry((-1,0)))
    lead1= kwant.Builder(kwant.TranslationalSymmetry((0,1)))
    lead2= kwant.Builder(kwant.TranslationalSymmetry((0,1)))

    def shape_lead0(site):
        x, y = site.pos
        return 0 < y <= width

    def shape_lead1(site):
        x, y = site.pos
        return width_lead < x <= 2*width_lead

    def shape_lead2(site):
        x, y = site.pos
        return 3*width_lead < x <= 4*width_lead

    lead0.fill(leads_model, shape_lead0, (0, 1))
    lead1.fill(leads_model, shape_lead1, (width_lead+1, 0))
    lead2.fill(leads_model, shape_lead2, (3*width_lead+1, 0))

    ########### create the six-terminal device
    for lead in [lead0,lead1,lead2, lead0.reversed(),lead2.reversed(),lead1.reversed()]:
        syst.attach_lead(lead)

    return syst.finalized()

In [2]:
def find_voltages(tmatrix,current):
    try:
        voltage = np.linalg.solve(tmatrix[1:, 1:], current[1:])
    except np.linalg.LinAlgError as err:
        if 'Singular matrix' in str(err):
            print("Hi there, singular matrix here")
            voltage = [0,0,0,0,0]
        else:
            raise
    return [0, *voltage]

In [71]:
syst=make_syst(width=60,length=300,leads_model=model_MTI())

m_value = 90
tmatrix = kwant.smatrix(syst,0.001970943,params=dict(m=m_value))
conductance_mat = tmatrix.conductance_matrix()
c=np.round(conductance_mat, 2)
X = np.abs(tmatrix.data)**2

print(tmatrix)
print(X)
print(c)

SMatrix(data=array([[ 0.68572501-0.1783616j , -0.29997481-0.09611523j,
        -0.09595373+0.3003262j , -0.31501556-0.01760814j,
         0.31152554-0.05211348j, -0.05241859-0.31176482j],
       [-0.16058038+0.27232112j,  0.10808715-0.70036417j,
         0.25527045+0.18458705j, -0.11477584+0.29360442j,
         0.04749361-0.31199385j, -0.31231601-0.04727842j],
       [-0.27194177-0.16065954j, -0.25623364-0.18520696j,
         0.10808715-0.70036417j, -0.29324354-0.11491474j,
         0.31167197+0.04770866j,  0.04749327-0.3119939j ],
       [ 0.30810625+0.06793677j,  0.30082913+0.09612365j,
         0.09596212-0.30118131j, -0.64831136+0.28612776j,
        -0.31069409+0.05163843j,  0.05194301+0.3109327j ],
       [ 0.26655487-0.16837925j,  0.28120481-0.14324701j,
        -0.14360934-0.28134133j,  0.23652105-0.2097065j ,
         0.10839751-0.7001412j ,  0.25549462+0.18427742j],
       [ 0.16800596+0.26645289j,  0.14288436+0.28106875j,
         0.28120497-0.1432467j ,  0.20932098+0.2364799

In [69]:
current= [-1, 0, 0, 1, 0, 0]


In [70]:
find_voltages(conductance_mat,current)

[0,
 np.float64(1.672259086916276),
 np.float64(1.672259463753525),
 np.float64(3.347987905454205),
 np.float64(1.6752925318656602),
 np.float64(1.675292155040104)]